In [1]:
import serial
import time
import pyvisa 
import numpy as np
from IPython.display import Image, display
from KeysightScope import KeysightScope

# You will have to change this to whatever COM port the pico is assigned when
# you plug it in.
# On Windows you can open device manager and look at the 'Ports (COM & LPT)' dropdown
# the pico will show up as 'USB Serial Device'
PICO_PORT = 'COM3'

MHZ = 1000000

conn = None
conn = serial.Serial(PICO_PORT, baudrate = 152000, timeout = 0.1)


## Helpers

In [2]:
my_instrument = KeysightScope()

In [ ]:
class BinaryRoutines():
    def __init__(self):
        self.mirror_channels = False

    def catch_response(self):
        resp = conn.readline().decode().strip()
        return resp
    
    def assert_OK(self):
        resp = conn.readline().decode().strip()
        assert resp == 'ok', 'Expected "ok", received "%s"' % resp

    def debug_on(self):
        conn.write(b'debug on\n')
        self.assert_OK()

    def set_mode(self, sweep_mode: int, timing_mode: int, num_channels: int):
        conn.write(b'reset\n')
        self.assert_OK()
        conn.write(b'mode %d %d \n' % (sweep_mode, timing_mode))
        self.assert_OK()
        conn.write(b'setchannels %d\n' % num_channels)
        self.assert_OK()
        print('sending commands to %d channels' % num_channels)

    def allocate(self, start_address: int, instructions: np.ndarray):
        conn.write(b'setb %d %d\n' % (start_address, len(instructions)))
        response = conn.readline().decode()
        if not response.startswith('ready'):
            response += ''.join([r.decode() for r in conn.readlines()])
            raise Exception(f'setb command failed, response: {repr(response)}')
        print(f'Got response: {response}')

    def table_to_memory(self, instructions: np.ndarray):
        conn.write(instructions.tobytes())
        self.assert_OK(), 'table not written correctly'
        print('table written to memory')

    def end_table(self, instructions: np.ndarray):
        conn.write(b'set 4 %d\n' % len(instructions))
        self.assert_OK(), 'table not stopped correctly'
        print('table end command sent')

    def start_routine(self):
        conn.write(b'start\n')
        self.assert_OK(), 'table did not start'
        print('table executed')

    def get_memory_layout(self):
        conn.write(b'getmode\n')
        mode = self.catch_response()
        channels = self.catch_response()
        memory_layout = self.catch_response()
        print(f"mode {mode}")
        print(channels)
        print(memory_layout)

binary_routine_test = BinaryRoutines()

In [4]:
## slightly different logic than the ad9959.c funcs due to python vs c
def get_ftw(freq_out: float, freq_sys: float = 500 * MHZ):

    ftw = (freq_out / freq_sys) * 2**32
    
    return(int(ftw))

def get_pow(phase: float):
    
    pow = round(phase / 360 * (2**14 - 1))

    return pow

def get_asf(amp: float):
    
    asf = round(amp * 1024)

    return asf

## Single Stepping Mode

Single Stepping (mode 0) with timing: `<frequency:int 32> <amplitude:int 16> <phase:int 16> <time: int 32>`. Total of 12 bytes per channel per instruction.

In [5]:
dt = np.dtype([('frequency', np.uint32), ('amplitude', np.uint16), ('phase', np.uint16), ('time', np.uint32)])

f1 = get_ftw(freq_out = 100 * MHZ)
f2 = get_ftw(freq_out = 120 * MHZ)
f3 = get_ftw(freq_out = 110 * MHZ)

p1 = get_pow(phase = 90)
p2 = get_pow(phase = 180)
p3 = get_pow(phase = 270)

t = 1

step_instructions = np.array([
                                (f2, 1023, p1, t),
                                (f2, 800, p1, t),
                                (f2, 1023, p1, t),
                                (f2, 200, p1, t),
                                (f2, 800, p1, t),
                                (f2, 1023, p1, t),
                                (f2, 200, p1, t),
                                (f2, 800, p1, t),
                                (f2, 1023, p1, t),
                                (f2, 800, p1, t)
                                 ]
                                 , dtype = dt)


In [6]:
single_step_test = BinaryRoutines()

In [11]:
single_step_test.set_mode(sweep_mode=0, timing_mode=1, num_channels=1)
single_step_test.allocate(start_address=0, instructions=step_instructions)
single_step_test.table_to_memory(instructions=step_instructions)
single_step_test.end_table(instructions=step_instructions)
single_step_test.start_routine()

sending commands to 1 channels
Got response: ready for 120 bytes

table written to memory
table end command sent
table executed


## Frequency Sweeps

Frequency Sweeps (mode 2) with timing: `<start frequency:int 32> <stop frequency:int 32> <delta:int 32> <rate:int 8> <time:int 32>`. Total of 17 bytes per channel per instruction.

In [12]:
dt = np.dtype([('start frequency', np.uint32), 
               ('stop frequency', np.uint32), 
               ('delta', np.uint32), 
               ('rate', np.uint8), 
               ('time', np.uint32)])

f1 = get_ftw(freq_out = 100 * MHZ)
f2 = get_ftw(freq_out = 120 * MHZ)
f3 = get_ftw(freq_out = 110 * MHZ)

d = 100
r = 1
t = 1

freq_sweep_table = np.array([
                            (f1, f3, d, r, t),
                            (f3, f2, d, r, t),
                            (f2, f1, d, r, t),
                            (f1, f1, d, r, t),
                            (f2, f2, d, r, t),
                            (f3, f3, d, r, t),
                            (f3, f1, d, r, t),
                            (f1, f2, d, r, t),
                            (f2, f3, d, r, t),
                            (f1, f1, d, r, t),
                                 ]
                                 , dtype = dt)


In [13]:
freq_sweep_test = BinaryRoutines()
freq_sweep_test.set_mode(sweep_mode=2, timing_mode=1, num_channels=1)
freq_sweep_test.allocate(start_address=0, instructions=freq_sweep_table)
freq_sweep_test.table_to_memory(instructions=freq_sweep_table)
freq_sweep_test.end_table(instructions=freq_sweep_table)
freq_sweep_test.start_routine()

sending commands to 1 channels
Got response: ready for 170 bytes

table written to memory
table end command sent
table executed


In [14]:
binary_routine_test.catch_response()

''